In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical 
import re

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reviews.csv")

In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
df.shape

(568454, 10)

In [ ]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(568411, 10)

In [ ]:
new_df = df[['Text','Score']]

In [ ]:
new_df.head()

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


In [ ]:
new_df.Score.unique()

array([5, 1, 4, 2, 3])

In [ ]:
new_df['Text'][4]

'Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.'

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#checking what all words are considered as stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#converting all the text to lower case
new_df['Text'] =  new_df['Text'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(new_df['Text'].iloc[3])

if you are looking for the secret ingredient in robitussin i believe i have found it.  i got this in addition to the root beer extract i ordered (which was good) and made some cherry soda.  the flavor is very medicinal.


In [ ]:
new_df.shape

(568411, 2)

In [ ]:
#loading the stopwords
stop_words = set(stopwords.words("english"))

In [ ]:
#removing the stop words from the corpus
new_df['Text'] = new_df['Text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(new_df['Text'].iloc[3])


looking secret ingredient robitussin believe found it. got addition root beer extract ordered (which good) made cherry soda. flavor medicinal.


In [ ]:
#tokenizing our document 
max_features = 5000
tokenizer = Tokenizer(num_words=max_features,split = ' ')
tokenizer.fit_on_texts(new_df['Text'].values)

In [ ]:
#converting our tokens or text to sequences
seq = tokenizer.texts_to_sequences(new_df['Text'].values)

In [ ]:
len(seq[3])

19

In [ ]:
print(seq[3])

[166, 2582, 497, 436, 47, 9, 75, 762, 1379, 1355, 1004, 122, 600, 3, 49, 827, 586, 10, 3608]


In [ ]:
print(seq[3])

[166, 2582, 497, 436, 47, 9, 75, 762, 1379, 1355, 1004, 122, 600, 3, 49, 827, 586, 10, 3608]


In [ ]:
#Doing the padding operation so that all the documents are of same length 
seq = pad_sequences(seq)

In [ ]:
print(seq[12])

[  0   0   0 ...  12 168  36]


In [ ]:
seq.shape

(568411, 1686)

In [ ]:
#converting the label to numeric form
y = pd.get_dummies(new_df['Score'],drop_first=True).values

In [ ]:
y[2]

array([0, 0, 1, 0], dtype=uint8)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(seq, y, test_size = 0.2,random_state = 0)

In [ ]:
x_train.shape

(454728, 1686)

In [ ]:
x_train[3]

array([   0,    0,    0, ..., 1556,  163, 1709], dtype=int32)

In [ ]:
y_train[3]


array([0, 0, 0, 1], dtype=uint8)

In [ ]:
embed_dimension = 128

In [ ]:
#Building the model 
model = Sequential()
model.add(Embedding(max_features,embed_dimension,input_length=seq.shape[1])) #Embedding layer - input_Dimension, embed_Dim we need, and the input length of the document
model.add(LSTM(162, dropout=0.2, recurrent_dropout= 0.2)) #buildinf a lstm layer 
model.add(Dense(1,activation = 'sigmoid')) #output layer

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1686, 128)         640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 162)               188568    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 163       
Total params: 828,731
Trainable params: 828,731
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#training our model and checking the validation accuracy
model.fit(x_train,y_train,batch_size=100,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10


ValueError: ignored